**Fitting Light curve**

In this note book, you try to fit the light curve.

Detect variabilities and Claim it statisticaly!

# Loading the *flute* output

We use PyROOT modules to load the output data.

If you intarested in ROOT, see https://root.cern/

In [ ]:
import ROOT
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit
import scipy.stats as stats
import numpy as np

In [ ]:
input_file = ROOT.TFile('/home/ssakurai/combine_lc_output.root')

In [ ]:
light_curve = input_file.LightCurve

In [ ]:
n_points = light_curve.GetN()

PyROOT is bit difficult... so data will be converted into numpy arrays in the below.

In [ ]:
x_value = []
y_value = []
x_error = []
y_error = []
x_tmp = light_curve.GetX()
y_tmp = light_curve.GetY()
for i in range(light_curve.GetN()):
        x_value.append(x_tmp[i])
        y_value.append(y_tmp[i])
        x_error.append(light_curve.GetErrorX(i))
        y_error.append(light_curve.GetErrorY(i))
x_value = np.array(x_value)
y_value = np.array(y_value)
x_error = np.array(x_error)
y_error = np.array(y_error)

Now you can accress data points and its errors through `x_value`, `y_value`, `x_error` and `y_error`

So.. the next is difining a fit rage.

# Defining a fit range

First, you should check your light curve by your eye.

Let's make a plot.

In [ ]:
fig = plt.figure(figsize=(10,5))
plt.plot([56243.95, 56244.11],[1.2e-10,1.2e-10],'b--',label='Crab flux > 300 GeV')
plt.errorbar(x_value,y_value,xerr=x_error,yerr=y_error,fmt='ko')
#plt.xlim(56243.96,56244.0) # change here and find range
#plt.ylim(0,1.5e-10) # change here if you need
plt.xlabel('Time (MJD)')
plt.ylabel('Flux > 300 GeV (???)')
plt.legend()
plt.show()

In [ ]:
your_fit_begin = 56244.030
your_fit_end =  56244.075

Once you dicide a fit range, let's try fitting by constant value at first.

# Fitting by a constant value

We will use *scipy* module to perform a fit.

the detail is here: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html

First you have to define a function what you want to fit.

So let's start from a constant function `your_func_c`
$$
    f(x) = C
$$

In [ ]:
def your_func_c(x, c):
    return x*0+c 

and for the simplisity, prepare the values only in the your fit range.

In [ ]:
index_fit = (x_value < your_fit_end) * (x_value > your_fit_begin)
x_fit = x_value[index_fit]
y_fit = y_value[index_fit]
ye_fit = y_error[index_fit]

In [ ]:
plt.errorbar(x_fit,y_fit,yerr=ye_fit,fmt='o')

In [ ]:
popt, pcov = curve_fit(your_func_c, x_fit,y_fit)

In [ ]:
print(popt)
print(np.sqrt(np.diag(pcov)))

In [ ]:
plt.errorbar(x_fit,y_fit,yerr=ye_fit,fmt='o')
x_func = np.arange(your_fit_begin,your_fit_end,0.001)
plt.plot(x_func,your_func_c(x_func,*popt),'k--')

And evaluate your fit by $\chi^2$

The followings are useful functions.

In [ ]:
def calculate_chi_square(x,y,yerr,func,param):
    n = len(x)
    tmp_array = [((y[i] - func(v,*param))/yerr[i])**2 for i,v in enumerate(x) ] 
    return np.sum(tmp_array)

In [ ]:
def calculate_dof(y,param):
    return len(y)-len(param)

In [ ]:
chi2_c = calculate_chi_square(x_fit,y_fit,ye_fit,your_func_c,popt)

In [ ]:
dof_c = calculate_dof(y_fit,popt)

From the degree of freedom and $\chi^2$, you can calculate the probability obtaining the observed data from assumed model. 

In [ ]:
stats.chi2.sf(chi2_c,dof_c)

So constatnt fit is not good. let's try another function.

# Fitting by your function

There are several functions to characterize a variavility. 

one example is an exponential curve.

$$
f(x) = C + a_0 \exp{\frac{(x-a_1)}{a_2}}
$$

Please define a function as you wish

In [ ]:
def your_func(x, a,b,c,d):
    return a+b*np.exp((x-c)/d) # This is an example!!! 

In [ ]:
#p0_init = 
#p1_init =
#p2_init =
#p3_init =
#p4_init =
popt, pcov = curve_fit(your_func, x_fit,y_fit)
#popt, pcov = curve_fit(your_func, x_fit,y_fit,p0=[p0_init,p1_init,p2_init,p3_init,p4_init])

In [ ]:
print('Parameters:',popt)
print('Errors:',np.sqrt(np.diag(pcov)))

In [ ]:
plt.errorbar(x_fit,y_fit,yerr=ye_fit,fmt='o')
x_func = np.arange(your_fit_begin,your_fit_end,0.001)
plt.plot(x_func,your_func(x_func,*popt),'k--')

In [ ]:
chi2_new = calculate_chi_square(x_fit,y_fit,ye_fit,your_func,popt)

In [ ]:
dof_new = calculate_dof(y_fit,popt)

In [ ]:
stats.chi2.sf(chi2_new,dof_new)

How was your fitting? If it does not work well, change a fit range or function!

Once you get the time scale of variability, let's convert to min or second